In [1]:
import os
import openai
from langchain.llms import AzureOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from dotenv import load_dotenv

load_dotenv()

True

In [6]:
llm = AzureOpenAI(
    openai_api_base=os.getenv('OPENAI_API_BASE'),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
    deployment_name='gpt-35-turbo-instruct',
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_type=os.getenv("OPENAI_API_TYPE"),
    temperature = 0.3
)

In [7]:
database_user = os.getenv("DATABASE_USERNAME")
database_password = os.getenv("DATABASE_PASSWORD")
database_server = os.getenv("DATABASE_SERVER")
database_db = os.getenv("DATABASE_DB")

connection_string = f"mssql+pymssql://{database_user}:{database_password}@{database_server}.database.windows.net:1433/{database_db}"

db = SQLDatabase.from_uri(connection_string, schema='SalesLT')
toolkit = SQLDatabaseToolkit(db=db, llm=llm, reduce_k_below_max_tokens=True)

In [7]:
toolkit.get_tools()

[QuerySQLDataBaseTool(name='sql_db_query', description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', using sql_db_schema to query the correct table fields.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, db=<langchain.utilities.sql_database.SQLDatabase object at 0x000002BD2EB1BFA0>),
 InfoSQLDatabaseTool(name='sql_db_schema', description="Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: 'table1, table2, table3'", args_schema=None, return_direct=False, verbose=False, callbacks=None, cal

In [8]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [5]:
agent_executor.run("How many people are there?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Thought: There are a lot of tables in this database.  I should look at the schema of some of the tables to see if I can find information about people.
Action: sql_db_schema
Action Input: Address, Customer
Observation: 
CREATE TABLE [SalesLT].[Address] (
	[AddressID] INTEGER NOT NULL IDENTITY(1,1), 
	[AddressLine1] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[AddressLine2] NVARCHAR(60) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[City] NVARCHAR(30) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[StateProvince] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[CountryRegion] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[PostalCode] NVARCHAR(15) COLLATE SQL_Latin1_General_CP1_CI_AS NOT

'There are 847 people in the database.'

In [10]:
agent_executor.run("How much money did each customer spend?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Address, Customer, CustomerAddress, Product, ProductCategory, ProductDescription, ProductModel, ProductModelProductDescription, SalesOrderDetail, SalesOrderHeader
Thought: I should query the schema for the Customer and SalesOrderHeader tables.
Action: sql_db_schema
Action Input: Customer, SalesOrderHeader
Observation: 
CREATE TABLE [SalesLT].[Customer] (
	[CustomerID] INTEGER NOT NULL IDENTITY(1,1), 
	[NameStyle] BIT NOT NULL DEFAULT ((0)), 
	[Title] NVARCHAR(8) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[FirstName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[MiddleName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[LastName] NVARCHAR(50) COLLATE SQL_Latin1_General_CP1_CI_AS NOT NULL, 
	[Suffix] NVARCHAR(10) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[CompanyName] NVARCHAR(128) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	[SalesPerson] NVARCHAR(256) COLLATE

"[(29736, Decimal('119960.8240')), (30050, Decimal('108597.9536')), (29546, Decimal('98138.2131')), (29957, Decimal('92663.5609')), (29796, Decimal('86222.8072')), (29929, Decimal('81834.9826')), (29932, Decimal('70698.9922')), (29660, Decimal('63686.2708')), (29938, Decimal('45992.3665')), (29485, Decimal('43962.7901'))]"